<a href="https://colab.research.google.com/github/nicolinux72/codellama-locally-with-gradio-and-huggingface/blob/main/codellama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codellama locally with gradio and huggingface
*A primer to AI applications with the aim of writing an open source, on premises personal code assistant.*

Please read full article ad [medium.com](https://medium.com/@nicolasanti_43152/local-code-llama-with-gradio-and-hugging-face-1153112046ec)

Let's start by installing the libraries we will use by running the following command.

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-z_l09cwi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-z_l09cwi
  Resolved https://github.com/huggingface/transformers.git to commit ef10dbce5cbc9a8b6a0a90b04378ca96f4023aa1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7606202 sha256=5d313207ba32305a2de77aa521c93c758bcc24e6d3a910faa920179994a6a55f
  Stored in directory: /tm

We can conclude the application setup by logging in.

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The transformers library is very concise and easy to use: the first step is to download the model, in our case denoted "codellama/CodeLlama-7b-hf".

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
pipeline = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Here is a quick example of tokenization that you can try with two lines.


In [4]:
text = "Hello, you are you?"
print (tokenizer.tokenize(text))


['▁Hello', ',', '▁you', '▁are', '▁you', '?']


Once the wait to download the model (only required for the first run) is over, we can directly use the pipeline object to make inference, which means asking our AI for something in order to get an answer.


In [5]:
sequences = pipeline(
    'def file_2_str(',
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    num_return_sequences=5,
    eos_token_id=tokenizer.eos_token_id,
    max_length=100,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: def file_2_str(self, file_path):
        with open(file_path, 'r') as f:
            return f.read()

    def file_2_list(self, file_path):
        with open(file_path, 'r') as f:
            return f.read().splitlines()

    def file_2_dict(self, file_path):
        with open(file_path
Result: def file_2_str(file_path):
    with open(file_path, 'r') as f:
        return f.read()


def file_2_list(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()


def file_2_dict(file_path):
    with open(file_path, 'r') as f:

Result: def file_2_str(self, file_path):
        with open(file_path, 'r') as f:
            return f.read()

    def file_2_list(self, file_path):
        with open(file_path, 'r') as f:
            return f.readlines()

    def file_2_dict(self, file_path):
        with open(file_path, '
Result: def file_2_str(self, file_path):
        with open(file_path, 'r') as f:
            return f.read()

    def file_2_list(self, file_path):
        

If the operation stalls due to the absence of the UTF-8 locale, as happens in colab, proceed to install it.

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

Gradio is a widely used project (21k stars on github and 1.5k forks) that allows you to set up an application with a handful of lines of code. We install it with the same pip command:

In [7]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 

Configure and run.

In [9]:
import gradio as gr


def predict(prompt):

  sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    num_return_sequences=5,
    eos_token_id=tokenizer.eos_token_id,
    max_length=100,
  )

  response = ''
  for seq in sequences:
    response += seq['generated_text']

  return response


description = """ <p>A short example of code helper bot that could run locally.</p>
"""

demo = gr.Interface(
  fn=predict,
  inputs=gr.Textbox(label="Please, write your request here:", placeholder="example: def fibonacci(", lines=5),
  outputs=gr.Textbox(label="Answer (inference):"),
  title='On Premesis Code LLama2 Helper',
  description=description,
  article='My article on Mediuam https://medium.com',
  examples=[["def Fibonacci("], ["function DotProduct("], ['springboot profile'], ['write a class for manage shipment']],
  allow_flagging="never"
)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>